In [12]:
import numpy as np
import pandas as pd

import nltk


In [13]:
train=pd.read_csv('train.csv')

In [14]:
test= pd.read_csv('test.csv')

In [15]:
train= train.drop_duplicates(subset='text', keep=False)
corpus= train['text'].to_list()
# print(corpus)

In [16]:
def pre_process(sentence):
    
    stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    
    sentence = sentence.lower()
    
   
    sentence= sentence.split()
    sentence= [word for word in sentence if word not in stopwords]
    sentence=[ ele for ele in sentence if ele not in punc]
    sentence=(' ').join(sentence)
   
    return sentence

In [17]:
corpus=[pre_process(sentence) for sentence in corpus]

In [18]:
labels= np.array(train['target'].to_list())

In [19]:
print(labels)

[1 1 1 ... 1 1 1]


In [20]:
train_sentences, test_sentences=corpus[ 0: 7000], corpus[ 7000: :]

train_labels, test_labels= labels[ 0: 7000], labels[ 7000: :]

In [21]:
vocab_size = 20000
max_length=None
embedding_dim = 16
trunc_type='post'
padding= 'post'
oov_tok = "<OOV>"

In [22]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import tensorflow as tf

In [23]:
tokenizer= Tokenizer(num_words=vocab_size,oov_token=oov_tok)

In [24]:
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index


In [25]:
train_sequences=tokenizer.texts_to_sequences(train_sentences)
train_padded= pad_sequences(train_sequences, dtype='int32',padding=padding,value=0.0)

test_sequences=tokenizer.texts_to_sequences(test_sentences)
test_padded= pad_sequences(test_sequences, maxlen=max_length,dtype='int32',padding=padding,value=0.0)

In [26]:
lstm_dim = 200
dense_dim=20

In [45]:
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=train_padded.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim, dropout=0.2)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

checkpoint=tf.keras.callbacks.ModelCheckpoint('keras_model',save_best_only=True, monitor='val_loss')
model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model_lstm.summary()



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 29, 16)            320000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 400)              347200    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 20)                8020      
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 675,241
Trainable params: 675,241
Non-trainable params: 0
_________________________________________________________________


In [32]:
NUM_EPOCHS = 10
BATCH_SIZE = 128


history_lstm = model_lstm.fit(train_padded, train_labels, batch_size=128, epochs=10, callbacks=[checkpoint])

Epoch 1/10
55/55 [==============================] - 8s 136ms/step - loss: 0.1902 - accuracy: 0.9280
Epoch 2/10
55/55 [==============================] - 7s 119ms/step - loss: 0.1209 - accuracy: 0.9577
Epoch 3/10
55/55 [==============================] - 7s 123ms/step - loss: 0.0720 - accuracy: 0.9750
Epoch 4/10
55/55 [==============================] - 7s 123ms/step - loss: 0.0621 - accuracy: 0.9794
Epoch 5/10
55/55 [==============================] - 7s 123ms/step - loss: 0.0437 - accuracy: 0.9876
Epoch 6/10
55/55 [==============================] - 7s 120ms/step - loss: 0.0382 - accuracy: 0.9901
Epoch 7/10
55/55 [==============================] - 6s 117ms/step - loss: 0.0291 - accuracy: 0.9920
Epoch 8/10
55/55 [==============================] - 7s 118ms/step - loss: 0.0264 - accuracy: 0.9930
Epoch 9/10
55/55 [==============================] - 7s 119ms/step - loss: 0.0218 - accuracy: 0.9947
Epoch 10/10
55/55 [==============================] - 7s 119ms/step - loss: 0.0164 - accuracy: 0.9953

In [49]:
from keras.models import load_model 
model = load_model('Keras_model')
test_sentences=test['text'].to_list()
test_sentences=[pre_process(sentence) for sentence in test_sentences]

OSError: No file or directory found at Keras_model

In [35]:
test_sequences=tokenizer.texts_to_sequences(test_sentences)
test_padded= pad_sequences(test_sequences, maxlen=max_length,dtype='int32',padding=padding,value=0.0)

In [51]:
prediction = model_lstm.predict(test_padded)

102/102 [==============================] - 4s 29ms/step


In [52]:
prediction

array([[0.4999969 ],
       [0.50154275],
       [0.50049436],
       ...,
       [0.50100166],
       [0.50121   ],
       [0.500652  ]], dtype=float32)

In [53]:
prediction[prediction>=0.5]=1
prediction[prediction<0.5]=0

In [55]:
sample_submission=pd.read_csv('sample_submission.csv')

In [56]:
sample_submission['target']=prediction
sample_submission['target']=sample_submission['target'].astype(int)

In [57]:
sample_submission.to_csv('submission.csv', index=False)